In [8]:
import os, sys, glob
import matplotlib.pyplot as plt

from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import precision_recall_fscore_support
from pathlib import Path
from src.utils import get_proj_root
import pandas as pd
import numpy as np
import detect_modality, find_thresholds, create_synthetic_data
# import data_loader_latency
from sklearn.model_selection import train_test_split
from tsfresh import extract_features, extract_relevant_features, select_features
from tsfresh.utilities.dataframe_functions import impute
from tsfresh.feature_extraction import MinimalFCParameters
from scipy import sparse
import tsfresh
import matplotlib.pyplot as plt
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import precision_recall_fscore_support
import pu_learn_fit as plf
import data_loader_latency as dl

from pulearn import (
    ElkanotoPuClassifier,
    # WeightedElkanotoPuClassifier,
)

import explore_anonymize_caida
import warnings

warnings.filterwarnings("ignore")
random_seed = 222

In [2]:
proj_root = get_proj_root()
path_proj = proj_root 
path_proj_pu = proj_root / 'project'
# path_caida = proj_root / 'pu_caida' / 'caida_datasets'
input_dir = path_proj / 'caida_processed'
# thres_dir = path_proj_pu / 'caida_processed'
current_dir = path_proj 

#Creating a folder if it does not exist
output_dir = path_proj_pu / 'caida_processed'
if not os.path.exists(output_dir):
    os.makedirs(output_dir)

In [3]:
# go to the original caida datasets
os.chdir(input_dir)
# collect the caida_data in the csv format
caida_data = glob.glob('*.csv')
print("caida data: ", caida_data)
# sort the dataset name to make it easier to track
caida_sorted = sorted(caida_data)
print("caida sorted: ", caida_sorted)
# # example use only the first dataset
# datasets = caida_sorted[0]
# print("datasets: ", datasets)
# anon_caida_name = []

caida data:  ['dataset_023.csv', 'dataset_018.csv', 'dataset_011.csv', 'dataset_016.csv', 'dataset_007.csv', 'dataset_000.csv', 'dataset_009.csv', 'dataset_017.csv', 'dataset_010.csv', 'dataset_019.csv', 'dataset_022.csv', 'dataset_008.csv', 'dataset_001.csv', 'dataset_006.csv', 'dataset_005.csv', 'dataset_002.csv', 'dataset_013.csv', 'dataset_014.csv', 'dataset_021.csv', 'dataset_003.csv', 'dataset_004.csv', 'dataset_020.csv', 'dataset_015.csv', 'dataset_012.csv']
caida sorted:  ['dataset_000.csv', 'dataset_001.csv', 'dataset_002.csv', 'dataset_003.csv', 'dataset_004.csv', 'dataset_005.csv', 'dataset_006.csv', 'dataset_007.csv', 'dataset_008.csv', 'dataset_009.csv', 'dataset_010.csv', 'dataset_011.csv', 'dataset_012.csv', 'dataset_013.csv', 'dataset_014.csv', 'dataset_015.csv', 'dataset_016.csv', 'dataset_017.csv', 'dataset_018.csv', 'dataset_019.csv', 'dataset_020.csv', 'dataset_021.csv', 'dataset_022.csv', 'dataset_023.csv']


In [4]:
def label(x, thres):
    if x >= thres:
        y = -1
    else:
        y = 1
    return y

In [5]:
os.chdir(output_dir)

In [9]:
f1_all = []
pre_all = []
rec_all = []
for d in caida_sorted:
    os.chdir(output_dir / str(d[:-4]))
    print(str(d[:-4]))
    ds = glob.glob('dataset*s*.csv')
    print(ds)
    ct = 1
    
    for s in ds:

        data = pd.read_csv(s, names=['time','rtt','id'])
        data['time'] = data['time'].apply(lambda x: str(x))
        extraction_settings = MinimalFCParameters()
        data_extracted_feats = extract_features(data,\
                                            column_id='id', column_sort='time',\
                                            default_fc_parameters=extraction_settings,\
                                            impute_function=impute)
        X_list = data_extracted_feats.values.tolist()
        X = np.array(X_list)
        train_primitive_matrix, val_primitive_matrix, test_primitive_matrix, \
            train_original, val_original, test_original = dl.split_data_2520(X, data)
        os.chdir(output_dir / str(d[:-4]))
        
        thres_name = 'Threshold-s'+ str(ct) + '-' + str(d)
        print(thres_name)
        ct += 1
        df_threshold = pd.read_csv(thres_name)
        thres = df_threshold['Threshold'][0]
        
        # label test data
        test_ground = test_original['rtt'].apply(label, thres=thres)

        # label val data
        val_ground = val_original['rtt'].apply(label, thres=thres)
 
        X_train_new = plf.concatenate_train_val(train_primitive_matrix, val_primitive_matrix)
        y_train_init = np.full(len(train_original,), -1., dtype=int)
        y_train_new = plf.create_new_y_train(y_train_init, val_ground)
        f1, pre, rec = plf.calculate_pu_scores(y_train_new, X_train_new, test_primitive_matrix, test_ground)
        f1_all.append(f1)
        pre_all.append(pre)
        rec_all.append(rec)

dataset_000
['dataset_000-s1.csv', 'dataset_000-s2.csv']


Feature Extraction: 100%|██████████████████████████████| 160/160 [00:00<00:00, 400.37it/s]


Threshold-s1-dataset_000.csv
PU transformation in progress...
-------------------
Fitting PU classifier (using a random forest as an inner classifier)...
F1 score: 0.6980728051391862
Precision: 1.0
Recall: 0.5361842105263158
PU transformation in progress...
-------------------
Fitting PU classifier (using a random forest as an inner classifier)...
F1 score: 0.6518847006651884
Precision: 1.0
Recall: 0.48355263157894735
PU transformation in progress...
-------------------
Fitting PU classifier (using a random forest as an inner classifier)...
F1 score: 0.7063829787234043
Precision: 1.0
Recall: 0.5460526315789473
PU transformation in progress...
-------------------
Fitting PU classifier (using a random forest as an inner classifier)...
F1 score: 0.6428571428571429
Precision: 1.0
Recall: 0.47368421052631576
PU transformation in progress...
-------------------
Fitting PU classifier (using a random forest as an inner classifier)...
F1 score: 0.7008547008547009
Precision: 1.0
Recall: 0.539473

F1 score: 0.7226890756302521
Precision: 1.0
Recall: 0.5657894736842105
PU transformation in progress...
-------------------
Fitting PU classifier (using a random forest as an inner classifier)...
F1 score: 0.6909871244635193
Precision: 0.9938271604938271
Recall: 0.5296052631578947
PU transformation in progress...
-------------------
Fitting PU classifier (using a random forest as an inner classifier)...
F1 score: 0.6965811965811967
Precision: 0.9939024390243902
Recall: 0.5361842105263158
PU transformation in progress...
-------------------
Fitting PU classifier (using a random forest as an inner classifier)...
F1 score: 0.7525562372188139
Precision: 0.9945945945945946
Recall: 0.6052631578947368
PU transformation in progress...
-------------------
Fitting PU classifier (using a random forest as an inner classifier)...
F1 score: 0.7751004016064257
Precision: 0.9948453608247423
Recall: 0.6348684210526315
PU transformation in progress...
-------------------
Fitting PU classifier (using a r

F1 score: 0.731006160164271
Precision: 0.9726775956284153
Recall: 0.5855263157894737
PU transformation in progress...
-------------------
Fitting PU classifier (using a random forest as an inner classifier)...
F1 score: 0.7227926078028748
Precision: 0.9617486338797814
Recall: 0.5789473684210527
PU transformation in progress...
-------------------
Fitting PU classifier (using a random forest as an inner classifier)...
F1 score: 0.7689243027888445
Precision: 0.9747474747474747
Recall: 0.6348684210526315
PU transformation in progress...
-------------------
Fitting PU classifier (using a random forest as an inner classifier)...
F1 score: 0.7048832271762207
Precision: 0.9940119760479041
Recall: 0.5460526315789473
PU transformation in progress...
-------------------
Fitting PU classifier (using a random forest as an inner classifier)...
F1 score: 0.7342799188640975
Precision: 0.9576719576719577
Recall: 0.5953947368421053
PU transformation in progress...
-------------------
Fitting PU classif

...

F1 score: 0.9138149556400507
Precision: 0.9769647696476965
Recall: 0.8583333333333333
PU transformation in progress...
-------------------
Fitting PU classifier (using a random forest as an inner classifier)...
F1 score: 0.8865573770491803
Precision: 0.9868613138686131
Recall: 0.8047619047619048
PU transformation in progress...
-------------------
Fitting PU classifier (using a random forest as an inner classifier)...
F1 score: 0.8900455432661029
Precision: 0.9813486370157819
Recall: 0.8142857142857143
PU transformation in progress...
-------------------
Fitting PU classifier (using a random forest as an inner classifier)...
F1 score: 0.8958742632612967
Precision: 0.9956331877729258
Recall: 0.8142857142857143
PU transformation in progress...
-------------------
Fitting PU classifier (using a random forest as an inner classifier)...
F1 score: 0.890038809831824
Precision: 0.9745042492917847
Recall: 0.819047619047619
PU transformation in progress...
-------------------
Fitting PU classifi

F1 score: 0.9108402822322001
Precision: 0.9874826147426982
Recall: 0.8452380952380952
PU transformation in progress...
-------------------
Fitting PU classifier (using a random forest as an inner classifier)...
F1 score: 0.9028132992327366
Precision: 0.9751381215469613
Recall: 0.8404761904761905
PU transformation in progress...
-------------------
Fitting PU classifier (using a random forest as an inner classifier)...
F1 score: 0.8804204993429697
Precision: 0.9824046920821115
Recall: 0.7976190476190477
PU transformation in progress...
-------------------
Fitting PU classifier (using a random forest as an inner classifier)...
F1 score: 0.8874430709173715
Precision: 0.9784791965566715
Recall: 0.8119047619047619
PU transformation in progress...
-------------------
Fitting PU classifier (using a random forest as an inner classifier)...
F1 score: 0.8996138996138996
Precision: 0.9789915966386554
Recall: 0.8321428571428572
PU transformation in progress...
-------------------
Fitting PU classi

F1 score: 0.9003880983182406
Precision: 0.9858356940509915
Recall: 0.8285714285714286
PU transformation in progress...
-------------------
Fitting PU classifier (using a random forest as an inner classifier)...
F1 score: 0.9129052765416401
Precision: 0.9795361527967258
Recall: 0.8547619047619047
PU transformation in progress...
-------------------
Fitting PU classifier (using a random forest as an inner classifier)...
F1 score: 0.9244444444444445
Precision: 0.9904761904761905
Recall: 0.8666666666666667
PU transformation in progress...
-------------------
Fitting PU classifier (using a random forest as an inner classifier)...
F1 score: 0.911764705882353
Precision: 0.9848066298342542
Recall: 0.8488095238095238
PU transformation in progress...
-------------------
Fitting PU classifier (using a random forest as an inner classifier)...
F1 score: 0.9037227214377407
Precision: 0.9805013927576601
Recall: 0.8380952380952381
PU transformation in progress...
-------------------
Fitting PU classif

F1 score: 0.9056122448979592
Precision: 0.9752747252747253
Recall: 0.8452380952380952
PU transformation in progress...
-------------------
Fitting PU classifier (using a random forest as an inner classifier)...
F1 score: 0.9066323245331616
Precision: 0.9873772791023843
Recall: 0.8380952380952381
PU transformation in progress...
-------------------
Fitting PU classifier (using a random forest as an inner classifier)...
F1 score: 0.9075738125802312
Precision: 0.9846796657381616
Recall: 0.8416666666666667
PU transformation in progress...
-------------------
Fitting PU classifier (using a random forest as an inner classifier)...
F1 score: 0.9104859335038363
Precision: 0.9834254143646409
Recall: 0.8476190476190476
PU transformation in progress...
-------------------
Fitting PU classifier (using a random forest as an inner classifier)...
F1 score: 0.917617237008872
Precision: 0.981029810298103
Recall: 0.861904761904762
PU transformation in progress...
-------------------
Fitting PU classifie

KeyboardInterrupt: 

In [10]:
methods = tuple([x[0] for x in sorted_f1])

NameError: name 'sorted_f1' is not defined